In [7]:
import pandas as pd

df = pd.read_csv(r"C:\Users\user\data\tables\lesion_followup_curated_v4.csv", sep=';')

In [9]:
df.groupby('patient').lung.max()

patient
IM_001     1
IM_002     0
IM_004     0
IM_012     1
IM_013     1
          ..
UNI_186    1
UNI_191    1
UNI_206    1
UNI_219    1
UNI_220    1
Name: lung, Length: 620, dtype: int64

In [12]:
from monai.transforms import LoadImaged, EnsureChannelFirstd, Compose
from src.transforms import (
    FindCentroid,
    GetFixedROISize,
    CropToCentroidWithSize,
    PadForCrop,
    Save,
    OrthogonalSlices,
    CropToROI,
    GetZoomedROI,
    PadForZoom,
)
from pathlib import Path

def find_corresponding_scan(segmention_path):
    scan_folder = Path(segmention_path).parent.parent / "scans"
    scan_name = Path(segmention_path).name.split(".")[0][:-2] + ".nii.gz"

    return scan_folder / scan_name

pipeline = Compose([
    LoadImaged(keys=["img", "seg"]),
    EnsureChannelFirstd(keys=["img", "seg"]),
])


segmentation_path = r'D:\premium_data\lumc\monotherapy\split_segmentations\PREM_LU_009_0.nii.gz'
scan_path = find_corresponding_scan(segmentation_path)
data = {'img':scan_path, 'seg':segmentation_path}

output = pipeline(data)


In [13]:
output['img'].shape

(1, 512, 512, 668)

In [14]:
from src.transforms import GetZoomedROI, PadForZoom, CropToROI

select = Compose(
    [
        GetZoomedROI(0),
        # PadForZoom(),
        # CropToROI()
    ]
)

roi = select(output)

In [15]:
roi['roi_start']

array([210, 257, 522])

In [16]:
roi['roi_end']

array([222, 269, 513])

In [22]:
import numpy as np

w = np.where(roi['seg'])[1:]
bbox_min_coordinates = np.array([axis.min() for axis in w])
bbox_max_coordinates = np.array([axis.max() for axis in w])



In [3]:
from pathlib import Path
import nibabel as nib

r  = Path(r'C:\Users\user\data\dl_radiomics\dim-2_size-256_method-crop_roi_size-50')

for c in r.iterdir():
    for f in c.iterdir():
        shape = nib.load(f).shape
        if not shape == (3,256,256):
            print(f.name, shape)

In [ ]:
dm = Data

In [1]:
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer, seed_everything
from src.model import Model
from src.data import DataModule
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

wandb.init()

wandb.config.aggregation_function = 'max'
wandb.config.method = 'crop'
wandb.config.dim = 2
wandb.config.size = 256
wandb.config.roi_size= 50
wandb.config.test_center = 'amphia'
wandb.config.lesion_target = 'lung'
wandb.config.max_batch_size = 12
wandb.config.seed = 0
wandb.config.max_epochs = 100
wandb.config.patience = 20
wandb.config.lr = 5e-4,
wandb.config.lr_min = 1e-6,
wandb.config.t_max = 20,

dm = DataModule(
    r"C:\Users\user\data\dl_radiomics",
    r"C:\Users\user\data\tables\lesion_followup_curated_v5.csv",
    wandb.config
)

# model = Model(wandb.config)

# seed_everything(wandb.config.seed)

# logger = WandbLogger(
#     name="hello5",
#     project="project_skeleton_on_lung_lesions",
# )

# checkpoint_callback = ModelCheckpoint(
#     monitor='valid_patient_auc',
#     mode='max'
# )

# early_stopping = EarlyStopping(monitor='valid_patient_auc', mode='max', patience=wandb.config.patience)

# trainer = Trainer(
#     max_epochs=wandb.config.max_epochs, gpus=1, deterministic=True, fast_dev_run=False, logger=logger,
#     checkpoint_callback=checkpoint_callback,
#     callbacks=[
#         early_stopping
#     ]
# )

# trainer.fit(model, dm)

2022-11-15 11:38:49,764 - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: renstermaat. Use `wandb login --relogin` to force relogin


In [3]:
dm.setup()
dl = dm.train_dataloader()

for _ in range(len(dl)):
    try:
        x = next(iter(dl))
    except Exception as e:
        print(e)
        break

Loading dataset: 100%|██████████| 168/168 [00:00<00:00, 167972.13it/s]


In [4]:
for img in dm.train_dataset:
    print(img['img'].shape)

(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 25

KeyboardInterrupt: 

In [5]:
from pathlib import Path
import os

r = Path(r'D:\premium_data\maxima\monotherapy\split_segmentations')

for file in r.iterdir():
    os.rename(file, file.parent / file.name.replace('.seg.nrrd',''))

In [3]:
wandb.config.t_max

[20]